In [ ]:
from creme import compose
from creme import linear_model
from creme import preprocessing


def parse(row):
    import datetime as dt
    row['date'] = dt.datetime.fromisoformat(row['date']).timestamp()
    return row

model = compose.FuncTransformer(parse) \
    | compose.Select('date', 'new_deceased', 'new_recovered', 'new_tested',
                     'cumulative_confirmed', 'cumulative_deceased',
                     'cumulative_recovered', 'cumulative_tested') \
    | preprocessing.StandardScaler() \
    | linear_model.LinearRegression()


In [ ]:
import dill
import requests

requests.post('http://localhost:5000/api/model', data=dill.dumps(model))

In [ ]:
!wget https://storage.googleapis.com/covid19-open-data/v3/epidemiology.csv

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('epidemiology.csv')

In [ ]:
for key in data.location_key.unique():
    if "DE" in key or "GER" in key:
        print(key)

In [ ]:
len(data.location_key.unique())